# Chroma Vector Database
- Chroma는 대규모 언어 모델(LLM) 애플리케이션 구축을 위해 설계된 AI 네이티브 **오픈 소스 벡터 데이터베이스**다.    
- 임베딩 저장소, 쿼리 및 검색 등의 핵심 기능을 제공하여 개발자들이 효율적으로 작업할 수 있도록 돕는다. 
- https://www.trychroma.com/
  
## Chroma의 주요 특징

- **오픈 소스 라이선스** 
  - Apache 2.0 라이선스에 따라 제공되어 누구나 자유롭게 사용하고 수정할 수 있다. 
- **다양한 개발 환경 지원**
  -  Python 및 JavaScript/TypeScript SDK를 지원하여 다양한 Langchain 과 연동하여 활용할 수 있다. 
- **유연한 데이터 저장 옵션**
  -  HTTP 방식, 디스크 저장 방식, 인메모리 방식을 선택하여 데이터를 저장할 수 있어 사용자 입장에서 매우 편리하다. 
- **간편한 사용법** 
  - 설치 및 사용법이 매우 간단하여 빠르게 프로토타입을 개발하고 검증할 수 있다. 

## 설치
- <del>pip로 chromadb 설치시 **windows**에서는 c컴파일러 관련되어 에러가 난다. **conda 를 이용해 설치한다.**<del>
- `conda install conda-forge::chromadb`
- `pip install chromadb`
- `pip install langchain-chroma`

In [1]:
!pip install chromadb

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
   ---------------------------------------- 0.0/19.3 MB ? eta -:--:--
   ----------- ---------------------------- 5.5/19.3 MB 30.5 MB/s eta 0:00:01
   ------------------------- -------------- 12.3/19.3 MB 30.8 MB/s eta 0:00:01
   ---------------------------------------  19.1/19.3 MB 31.0 MB/s eta 0:00:01
   ---------------------------------------- 19.3/19.3 MB 29.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 1.9/1.9 MB 26.7 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.73.0 requires protobuf<7.0.0,>=6.30.0, but you have protobuf 5.29.5 which is incompatible.


# Chroma API 를 이용해 연동
- https://docs.trychroma.com/

In [2]:
import chromadb

In [ ]:
from uuid import uuid4

str(uuid4()) # ID값 생성해주는 함수. / 계속변함

In [1]:
from uuid import uuid4

# 추가할 데이터
document_list = [
        "This is a document about pineapple",
        "This is a document about oranges",
        "This is a document about sports",
        "This is a document about langchain",
]
ids = [str(uuid4()) for _ in range(len(document_list))]
# DB에 저장할 때 지정할 각 문서들의 ID 생성.

In [2]:
from langchain_openai import OpenAIEmbeddings

# Embedding modle 생성.
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [7]:
# 외부 Embedding model 사용

import chromadb.utils.embedding_functions as embedding_functions
from dotenv import load_dotenv
import os

print(load_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPENAI_API_KEY,
                model_name="text-embedding-3-small"
            )

True


In [8]:
# collection 생성 - Database 생성
# Chroma DB와 연결

import chromadb

client = chromadb.Client() # Inmemory DB 생성. -> 데이터를 메모리에 저장.
# client = chromadb.PersistentClient(path="vector_store/my_db") # Local 파일에 저장
# client = chromadb.HttpClient(host="ip주소", port=8000) # 서버로 서비스하는 chromadb에 연결.

# Collection(Database)를 생성.
collection_name = "test_db"
collection = client.create_collection(
    name=collection_name,
    get_or_create=True, # default = False / collection 이 있으면 연결, 없으면 생성. / False이면 동일한 collection이 존재 할시 exception.
    metadata={"hnsw:space":"cosine"},
    embedding_function=openai_ef
)

In [9]:
#  데이터 추가

collection.add(documents=document_list, ids=ids)

In [10]:
# 유사도 겁색

result = collection.query(
    query_texts= ["deeplearning", "hawaii"], # 질문
    n_results=2, # 검색 결과의 개수
)

result

{'ids': [['22d6b42f-fff7-4070-9c18-3a9f2a1a9767',
   '663b1671-9785-498b-93b7-181547eb7b6e'],
  ['663b1671-9785-498b-93b7-181547eb7b6e',
   '22d6b42f-fff7-4070-9c18-3a9f2a1a9767']],
 'embeddings': None,
 'documents': [['This is a document about langchain',
   'This is a document about pineapple'],
  ['This is a document about pineapple',
   'This is a document about langchain']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None], [None, None]],
 'distances': [[0.7781245708465576, 0.8145824670791626],
  [0.7556753754615784, 0.8807662725448608]]}

# Langchain을 이용해 Chroma 연동

## Data 준비

In [25]:
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
    id=5,
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
    id=6,
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
    id=7,
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
    id=8,
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)

document_list = [document_1, document_2, document_3, document_4, document_5, document_6, document_7, document_8, document_9, document_10]

ids = [str(uuid4()) for _ in range(len(document_list))]

## Vector Store 생성, 연결
- Chroma.from_documents()
  - VectorStore를 초기화(생성)하고 문서를 추가한다.
  - persist_directory를 지정하지 않으면 메모리에 저장된다.
- Chroma()
  - VectorStore와 연결.


In [27]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

COLLECTION_NAME = "example" # 컬렉션 이름.
PERSISTENT_PATH = "vector_store/chroma/example_db" # 저장할 로컬 경로.

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 1. 연결(생성)하면서 document들을 저장.(upsert)

vector_store = Chroma.from_documents(
    documents = document_list,
    ids = ids,
    embedding = embedding_model,
    collection_name = COLLECTION_NAME,
    persist_directory = PERSISTENT_PATH
)

## VectorStore 정보 확인

In [28]:
coll = vector_store._collection # 연결된 collection 정보를 확인
coll

Collection(name=example)

In [29]:
coll.count() # 저장된 데이터 개수

10

## Add (추가)

In [22]:
document_11 = Document(
    page_content="랭체인은 대규모 언어 모델(LLM)을 효과적으로 활용하기 위한 도구와 프레임워크를 제공하는 오픈소스 라이브러리입니다.",
    metadata={"source": "tweet"},
    id=10,
)

document_12 = Document(
    page_content="랭체인은 체인 구조를 사용하여 여러 LLM 작업을 연결하고, 이를 통해 더 복잡하고 맞춤화된 자연어 처리 애플리케이션을 개발할 수 있게 합니다",
    metadata={"source": "tweet"},
    id=10,
)

document_13 = Document(
    page_content="랭체인, AI 활용의 새 시대를 열다: 복잡한 언어 처리도 간단하게!",
    metadata={"source": "news"},
    id=10,
)

In [30]:
vector_store.add_documents(
    [document_11, document_12, document_13],
    ids = [str(uuid4()), str(uuid4()), str(uuid4())]
    )


['ba33483b-69ef-463b-bfe4-db7decde56e7',
 '7504ce84-d0b6-42f2-a871-5b8e880141e5',
 '4a8eccd6-f0b9-455e-abeb-6610a7d2253a']

In [31]:
coll.count()

13

## Update(갱신)

In [32]:
new_document_13 = Document(
    page_content="랭체인, AI 활용의 새 시대를 열다: 복잡한 언어 처리도 간단하게 처리할수 있는 Framework!",
    metadata={"source": "news"},
    id=10,
)

In [34]:
vector_store.update_document(
    document_id="4a8eccd6-f0b9-455e-abeb-6610a7d2253a", # 바꿀 문서의 ID
    document= new_document_13 # 바꿀 내용을 가진 Document 객체
)

In [43]:
update_document_12 = Document(
    page_content="랭체인은 체인 구조를 사용하여 여러 LLM 작업을 연결할 수있다.",
    metadata={"source": "website"},
)

update_document_13 = Document(
    page_content="랭체인, AI 활용의 새 시대를 열다: 복잡한 언어 처리도 간단하게!",
    metadata={"source": "news"},
)

update_docs = [update_document_12, update_document_13]
update_ids = ['7504ce84-d0b6-42f2-a871-5b8e880141e5', '4a8eccd6-f0b9-455e-abeb-6610a7d2253a']

# 한번에 여러개 update
vector_store.update_documents(documents=update_docs, ids=update_ids)


In [44]:
coll.get() # 전체 저장된 문서를 조회
# vector_store.get()

{'ids': ['428e603d-a8c2-418d-89ff-50ebdbdab8aa',
  'fefdadc9-5c77-4452-bc2a-91119440043c',
  '6a890808-a671-406f-9756-0318d1900341',
  '3d3d67a4-8f6d-484d-9a23-3e3c14934b47',
  '3425e544-f4fd-4608-9dad-c68f76cf4cc6',
  'c8435827-32a2-4b14-bdbe-791d50dda6a1',
  '97ca3bca-7944-4b2b-80c1-1e232f4139e7',
  '45191a18-1603-4056-9b24-38230bc0ab31',
  'cb40c318-80a9-4a9a-ac1c-a94d6b900f56',
  '2f4f853c-326e-43ca-9b90-ac3505abf9fe',
  'ba33483b-69ef-463b-bfe4-db7decde56e7',
  '7504ce84-d0b6-42f2-a871-5b8e880141e5',
  '4a8eccd6-f0b9-455e-abeb-6610a7d2253a'],
 'embeddings': None,
 'documents': ['I had chocolate chip pancakes and scrambled eggs for breakfast this morning.',
  'The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.',
  'Building an exciting new project with LangChain - come check it out!',
  'Robbers broke into the city bank and stole $1 million in cash.',
  "Wow! That was an amazing movie. I can't wait to see it again.",
  'Is the new iPhone worth the 

## Delete(삭제)

In [45]:
del_ids = ['6a890808-a671-406f-9756-0318d1900341', '3d3d67a4-8f6d-484d-9a23-3e3c14934b47']

vector_store.delete(ids=del_ids) # 삭제할 문서의 id들

In [46]:
coll.count()

11

## Query(조회)
- `similarity_search(query, k, filter)`
  - 저장되 있는 item들 중 질의와 가장 유사한 것 k개를 찾는다. 
  - 찾은 결과를 filter 조건으로 필터링 한다. filter 조건은 meta-data의 정보를 이용한다.
  - 질의어(query)는 text(자연어)로 입력한다.
- `similarity_search_with_score(query, k, filter)`
  - 저장되 있는 item들 중 질의와 가장 유사한 것 k개를 찾아 유사도 점수와 함께 반환
- `similarity_search_by_vector(embedding, k, filter)`
  - Embedding Vector 를 질의로 입력한다. (질의(query)를 문장이 아니라 embedding vector로 입력.) 

In [ ]:
results = vector_store.similarity_search_with_score(
    query = "아침에 뭐먹을까?",
    k = 3, # 조회 개수
    # filter= {"source":"tweet"} # metadata의 source키 값이 tweet(source == tweet)
    filter = {"source":{"$ne":"news"}} # source가 news가 아닌것들.
    # {metadata key: {"연산자":"값"}}
    # {"age":{"$gt":30}} -> age > 30
)
# 1. filter에 설정과 metadata를 비교해서 조회
# 2. 1에서 조회된 문서들과 query간의 유사도를 체크
# 필터에서 먼저거르고 k개수 만큼 찾기
results

[(Document(id='428e603d-a8c2-418d-89ff-50ebdbdab8aa', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
  1.1996257305145264),
 (Document(id='c8435827-32a2-4b14-bdbe-791d50dda6a1', metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.'),
  1.8273663520812988),
 (Document(id='ba33483b-69ef-463b-bfe4-db7decde56e7', metadata={'source': 'tweet'}, page_content='랭체인은 대규모 언어 모델(LLM)을 효과적으로 활용하기 위한 도구와 프레임워크를 제공하는 오픈소스 라이브러리입니다.'),
  1.8730052709579468)]